In [3]:
import pandas as pd

In [71]:
data = pd.read_csv('/data/word2vec/googlenews.txt', sep=' ', usecols=[0], header=None, skiprows=1)

In [1]:
data.count()

NameError: name 'data' is not defined

In [10]:
from load_data import *

In [11]:
datafolder = '/data/NYT/'
sourcefilename = 'nyt-freebase.train.triples.universal.mention.txt'

In [12]:
labels, relations, sentences, entAs, entBs = read_data(datafolder+sourcefilename)

In [28]:
pos_count = labels.count('POSITIVE')
neg_count = labels.count('NEGATIVE')
unl_count = labels.count('UNLABELED')

print 'data includes %i POSITIVE, %i NEGATIVE, %i UNLABELED.' % (pos_count, neg_count, unl_count)

data includes 23696 POSITIVE, 420188 NEGATIVE, 1073623 UNLABELED.


In [31]:
import pandas as pd

In [39]:
data = pd.DataFrame({'labels': labels, 'relations': relations,
                     'sentences': sentences, 'entAs': entAs, 'entBs': entBs})

In [54]:
pos = data[data['labels'] == 'POSITIVE']

In [67]:
neg = data[data['labels'] == 'NEGATIVE'].sample(n=pos_count)

In [70]:
data_filtered = pos + neg

In [ ]:
data_filtered